In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
import sys,os
sys.path.insert(0,'../')
import StatPol as SP

basepath = os.getcwd()
basepath

'/home/dalessandro/work/Statical_polarizability/Calculations'

In [2]:
omp = 1
mpi_run = 'mpirun -np 16'

# Workflow

This notebook contains the complete workflow to extract the statical polarizability of the HG dataset.

## Construction of the relevant tools

Define the functions needed to perform the workflow

In [3]:
# Service functions 

def change_workdir(molecule,study,datadir):
    """
    Ensure that the path Data_directory/molecule exists and
    move there. 
    """
    from futile.Utils import ensure_dir
    for d in [datadir,molecule]:
        ensure_dir(d)
        os.chdir(d)
    path='-'.join(study)
    ensure_dir(path)
    return '-'.join([molecule,path]),path

def get_psp(molecule):
    """
    Check if the all the atoms of molecule have the psp of the type nlss_aw and nlcc_ss. 
    If it is true the corresponding pseudo is added to the list of psp to be performed, otherwise 
    only hgh_k is included.
    """
    from os import path as p
    import Routines as R
    possible_psp=['nlcc_aw','nlcc_ss']
    to_do=[True,True]
    for at in R.get_atoms(molecule):
        psp_available = [p.isfile(p.join('psp_'+psp,'psppar.'+at)) for psp in possible_psp]
        to_do = [ a and b for a,b in zip(to_do,psp_available)]
    return ['hgh_k'] + [psp for yes,psp in zip(to_do,possible_psp) if yes]

def split_dataset(dataset,length=10):
    """
    Split a list of molecules into a list of list with length elements. 
    """
    splitted = [[] for num in range(int(len(dataset)/length)+1)]
    for ind,mol in enumerate(dataset):
        splitted[int(ind/length)].append(mol)
    return splitted

In [4]:
# Tools for the ground state

def get_converged_input_energy(dataset,rtol,atol):
    data=dataset.seek_convergence(atol=atol,rtol=rtol,attribute='energy')
    return dataset.runs[dataset.names.index(D.name_from_id(data[0]))]['input'],data[1]

def find_gs_domain(label,rtol,atol,path,input,posinp,code):
    """
    Use the seek_convergence method of Dataset to perform a convergence procedure on
    the rmult value for a gs computation.
    """
    crmult=input['dft']['rmult'][0]
    rmult_fine=input['dft']['rmult'][1]
    rmult_list=map(float,range(int(crmult),11))
    seek_for_rmult = D.Dataset(label=label+'(GS)',run_dir=path,posinp=posinp)
    for rm in rmult_list:
        input.set_rmult(coarse=rm,fine=rmult_fine)
        seek_for_rmult.append_run(id={'rmult':rm},runner=code,input=input)
    input_gs,log_gs=get_converged_input_energy(seek_for_rmult,rtol,atol)
    return {'input_gs':input_gs, 'dataset_gs': seek_for_rmult,'log_gs':log_gs}

def gs_study(molecule,study,code,options):
    """"
    Workflow for the convergence analysis of the ground state.
    
    Args : 
       molecule (str) : name of the molecule
       study (touple) : the couple (xc,psp)
       code (runner)  : instance of SystemCalculator
       options (dict) : dictionary with the computational options
    """
    import Routines as R
    hgrids=options.get('hgrids')
    rmult_fine=options.get('rmult_fine')
    wf_convergence=options.get('wf_convergence')
    crmult_start=options.get('crmult',4.0)
    rtol=options.get('rtol_gs',10*wf_convergence)
    atol=options.get('atol_gs',1.e-3)
    reference_results=options.get('reference_data')
    datadir=options.get('data_directory','Data')

    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,datadir)
    
    posinp=Molecules.Molecule(molecule)
    
    inp = I.Inputfile()
    inp.set_hgrid(hgrids)
    inp.set_rmult(coarse=crmult_start,fine=rmult_fine)
    R.set_xc[study](inp,molecule,datadir)
    inp.set_wavefunction_convergence(wf_convergence)
    mpol=int(reference_results[molecule]['mpol_ref'])-1   
    if mpol >0: inp.spin_polarize(mpol)
    data= find_gs_domain(label,10*wf_convergence,atol,path,inp,posinp,code)
    os.chdir(initial_dir)
    
    data['molecule']=molecule
    data['posinp']=posinp
    data['study']=study
    return data 

In [5]:
# Tools for alpha
def extract_statical_polarizability(label,rtol,atol,path,input_for_alpha,posinp,code,ref,single_field):
    
    # seek for the field convergence
    if single_field is not None:
        print 'Use field intensity : ' +str(single_field)
        intensity=single_field
        alpha_field=None 
    else:
        ints=[1e-2,5e-3,1e-3,5e-4,1e-4]
        alpha_field=D.Dataset(label=label+'(field)',run_dir=path)
        for f in ints:
            alpha_field.append_run(id={'f':f},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=f))
        try:
            data_field=alpha_field.seek_convergence(rtol=rtol,atol=atol)
            intensity=data_field[0]['f']
        except:
            print ('Convergence in field not reached')
            intensity=ints[-1]
    
    # seek for the rmult convergence
    rmult_fine=input_for_alpha['dft']['rmult'][1]
    rmult_list=map(float,range(int(input_for_alpha['dft']['rmult'][0]),13)) #enlarged for seeking convergence in some molecules
    alpha_rmult=D.Dataset(label=label+'(domain)',run_dir=path)
    for rm in rmult_list:
        input_for_alpha.set_rmult(coarse=rm,fine=rmult_fine)
        alpha_rmult.append_run(id={'f':intensity,'rmult':rm},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=intensity))
    try:
        import numpy as np
        data_final=alpha_rmult.seek_convergence(rtol=rtol,atol=atol)
        AuToA = 0.5291772085**3
        alpha_ref_ii = np.array(ref)/AuToA
        eps = 100.0 * (data_final[1]-alpha_ref_ii)/alpha_ref_ii
        print ('Relative difference in %',eps.tolist())
    except LookupError as e:
        print ('Convergence in domain not reached',e)
        data_final=None
    return {'alpha_convergence': data_final, 'dataset_field': alpha_field, 'dataset_rmult': alpha_rmult}

def alpha_study(gs_data,code,options,single_field=None):
    """"
    Workflow for the convergence analysis for the computation of alpha.
    
    Args : 
       gs_data        : output of gs_study
       code (runner)  : instance of SystemCalculator
       options (dict) : dictionary with the computational options
    """
    import copy
    xc_conversion={'lda_pw': 'lda-SPW92', 'pbe': 'pbe', 'pbe0': 'pbe0'}
    rtol=options.get('rtol',1.e-2)
    atol=options.get('atol',1.e-3)
    reference_results=options.get('reference_data')
    datadir=options.get('data_directory','Data')
    study=gs_data['study']
    molecule=gs_data['molecule']
    posinp=gs_data['posinp']
    xc=study[0]
    ref_data=reference_results[molecule][xc_conversion[xc]]
    
    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,datadir)
    input_gs=copy.deepcopy(gs_data['input_gs'])
        
    data=extract_statical_polarizability(label,rtol,atol,path,input_gs,posinp,code,ref=ref_data,single_field=single_field)
    os.chdir(initial_dir)
    
    return data

In [6]:
def data_to_save(data,options):
    """
    Define the dictionary to be saved on file for data analysis
    """
    from copy import deepcopy
    res = {}
    for s,values in data.iteritems():
        res[s] = {}
        if not (values['alpha_convergence'] is None):
            res[s]['alpha_convergence'] = (values['alpha_convergence'][0],values['alpha_convergence'][1].tolist())
        else : 
            res[s]['alpha_convergence'] = values['alpha_convergence']
        
        res[s]['posinp'] = {}
        pos = values['posinp']
        for key in pos:
            res[s]['posinp'][key] = pos[key]
        
        res[s]['input_gs'] = {}
        inp = values['input_gs']
        for key in inp:
            res[s]['input_gs'][key] = inp[key]
        
    save_options = deepcopy(options)
    save_options.pop('reference_data')
    res['options'] = save_options
    return res

## Single study analysis

Build the dataset using the HG data written in hg_data.yaml. To each molecule of the dataset associate a set of studies=(xc,psp)

In [29]:
import yaml
HG_data=yaml.load(open('../HG Dataset/hg_data.yaml'))

In [30]:
code=C.SystemCalculator(omp=omp,mpi_run=mpi_run,skip=True,verbose=False)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 16 /home/dalessandro/Applications/BigDFT/binaries/v1.8.4/install/bin/bigdft


In [31]:
options={'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-3,'atol':1.e-3,\
         'reference_data':HG_data,'data_directory':'Data'}

We can perform a single study with this workflow

In [32]:
molecule = 'C2H'
xc = 'pbe'
psp = 'hgh_k'
study_data = {}

In [33]:
gs_data=gs_study(molecule,(xc,psp),code,options)

Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Convergence reached in Dataset "C2H-pbe-hgh_k(GS)" for id " {'rmult': 5.0} "


In [34]:
gs_data

{'dataset_gs': <BigDFT.Datasets.Dataset instance at 0x7fecfa38f908>,
 'input_gs': {'dft': {'gnrm_cv': 1e-06,
   'hgrids': 0.3,
   'ixc': 11,
   'mpol': 1,
   'nspin': 2,
   'rmult': [5.0, 9.0]}},
 'log_gs': <BigDFT.Logfiles.Logfile instance at 0x7fecf5a7f950>,
 'molecule': 'C2H',
 'posinp': {'positions': [{'C': [0.0, 0.0, 0.0], 'frag': ['molecule', '0']},
   {'C': [0.0, 0.0, -1.2165], 'frag': ['molecule', '0']},
   {'H': [0.0, 0.0, 1.0465], 'frag': ['molecule', '0']}],
  'units': 'angstroem'},
 'study': ('pbe', 'hgh_k')}

In [35]:
s_f = 0.0005

In [36]:
study_data[(molecule,xc,psp)]=alpha_study(gs_data,code,options,single_field=s_f)
study_data[(molecule,xc,psp)].update(gs_data)

Use field intensity : 0.0005
Fetching results for id " {'rmult': 5.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 6.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 7.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 8.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 9.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 10.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 11.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 12.0, 'f': 0.0005} "
('Convergence in domain not reached', LookupError('Convergence not reached, enlarge the dataset or change tolerance values',))


In [37]:
study_data

{('C2H', 'pbe', 'hgh_k'): {'alpha_convergence': None,
  'dataset_field': None,
  'dataset_gs': <BigDFT.Datasets.Dataset instance at 0x7fecfa38f908>,
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7fecf8492830>,
  'input_gs': {'dft': {'gnrm_cv': 1e-06,
    'hgrids': 0.3,
    'ixc': 11,
    'mpol': 1,
    'nspin': 2,
    'rmult': [5.0, 9.0]}},
  'log_gs': <BigDFT.Logfiles.Logfile instance at 0x7fecf5a7f950>,
  'molecule': 'C2H',
  'posinp': {'positions': [{'C': [0.0, 0.0, 0.0], 'frag': ['molecule', '0']},
    {'C': [0.0, 0.0, -1.2165], 'frag': ['molecule', '0']},
    {'H': [0.0, 0.0, 1.0465], 'frag': ['molecule', '0']}],
   'units': 'angstroem'},
  'study': ('pbe', 'hgh_k')}}

In [38]:
data_rmult = study_data['C2H','pbe','hgh_k']['dataset_rmult']
data_rmult.results

{0: array([24.73516, 24.73516, 27.5758 ]),
 1: array([24.95816, 24.95816, 27.8219 ]),
 2: array([25.85046, 25.85046, 27.8672 ]),
 3: array([25.14882, 25.14882, 27.8727 ]),
 4: array([25.33919, 25.33919, 27.8771 ]),
 5: array([25.39615, 25.39615, 27.8762 ]),
 6: array([25.12342, 25.12342, 27.8769 ]),
 7: array([25.85049, 25.85049, 27.877  ])}

To check if convergence has been reached we can look at 'alpha_convergence'. 

If the value of 'f' is the last value of the field intensity list provided in the extract_statical_polarizability function, then the field convergence has not been reached. Moreover, if 'alpha_convergence' is None the rmult convergence on alpha has failed. 

In [16]:
data_to_save(study_data,options)

{'options': {'atol': 0.001,
  'data_directory': 'Data',
  'hgrids': 0.3,
  'rmult_fine': 9.0,
  'rtol': 0.001,
  'wf_convergence': 1e-06},
 ('CO',
  'pbe',
  'nlcc_aw'): {'alpha_convergence': ({'f': 0.005, 'rmult': 7.0},
   [12.540765999999998,
    12.540765999999998,
    15.889695000000001]), 'input_gs': {'dft': {'gnrm_cv': 1e-06,
    'hgrids': 0.3,
    'ixc': 11,
    'rmult': [5.0, 9.0]}}, 'posinp': {'positions': [{'C': [0.0, 0.0, 0.0],
     'frag': ['molecule', '0']},
    {'O': [0.0, 0.0, 1.1282], 'frag': ['molecule', '0']}],
   'units': 'angstroem'}}}

## Dataset analysis

In the same way we can run the whole dataset (or a part of it). The results of the dataset (processed by data_to_save) are stored in the full_data dictionary. This dictionary can be dumped in a Yaml file and updated with new computatitions so that the complete dataset can be runned in more than one step. 

In [7]:
import yaml
HG_data=yaml.load(open('../HG Dataset/hg_data.yaml'))
# set the study for H to sp
HG_data['H']['spin_pol'] = 'sp'

In [8]:
# for computational reason it can be useful to split the complete dataset into the sp and the nsp ones.
# Moreover, each dataset can be divided into groups. 
sp_dataset = []
nsp_dataset = []
for mol in HG_data:
    if HG_data[mol]['spin_pol'] == 'nsp': 
        nsp_dataset.append(mol)
    else : 
        sp_dataset.append(mol)

# remove Li2 because the gs calculation does not converge with the
# actual range of parameters
ind =  sp_dataset.index('Li2')
del sp_dataset[ind]

sp_dataset.sort()
nsp_dataset.sort()
sp_split = split_dataset(sp_dataset)
nsp_split = split_dataset(nsp_dataset)

In [9]:
print len(nsp_dataset)
print len(sp_dataset)

74
57


In [10]:
sp_split

[['BH2', 'BN', 'BO', 'BS', 'Be', 'BeH', 'C2H', 'C2H3', 'CH2-t', 'CH2F'],
 ['CH2NH', 'CH2PH', 'CH3', 'CH3O', 'CN', 'F2', 'FCO', 'FH-OH', 'H', 'H2CN'],
 ['H2O-Li', 'HCHS', 'HCO', 'HCP', 'HNO', 'HNS', 'HO2', 'HOF', 'Li', 'N'],
 ['N2H2', 'NCO', 'NH', 'NH2', 'NO', 'NOCl', 'NP', 'Na', 'Na2', 'NaLi'],
 ['O2', 'O3', 'OCl', 'OF', 'OF2', 'OH', 'P', 'P2', 'PH', 'PH2'],
 ['PS', 'S2', 'SCl', 'SF', 'SH', 'SO-trip', 'SiH3']]

We also build a complete dataset that can be run to write all the results on file

In [11]:
full_dataset = []
for mol in HG_data:
        full_dataset.append(mol)
# remove Li2 because the gs calculation does not converge with the
# actual range of parameters
ind =  full_dataset.index('Li2')
del full_dataset[ind]
full_dataset.sort()

In [12]:
len(full_dataset)

131

In [13]:
# check if some molecule folder is missing in Data
for mol in full_dataset:
    d = os.path.join('Data',mol)
    if not os.path.isdir(d):
        print mol

In [14]:
full_dataset_split = split_dataset(full_dataset)

In [15]:
for sets in full_dataset_split:
    print sets

['AlF', 'Ar', 'BF', 'BH2', 'BH2Cl', 'BH2F', 'BH3', 'BHF2', 'BN', 'BO']
['BS', 'Be', 'BeH', 'BeH2', 'C2H', 'C2H2', 'C2H3', 'C2H4', 'CH2-t', 'CH2BH']
['CH2F', 'CH2NH', 'CH2PH', 'CH3', 'CH3BH2', 'CH3Cl', 'CH3F', 'CH3NH2', 'CH3O', 'CH3OH']
['CH3SH', 'CH4', 'CN', 'CO', 'CO2', 'CS', 'CSO', 'Cl2', 'ClCN', 'ClF']
['F2', 'FCN', 'FCO', 'FH-OH', 'FNO', 'H', 'H2', 'H2CN', 'H2O', 'H2O-Li']
['HBO', 'HBS', 'HCCCl', 'HCCF', 'HCHO', 'HCHS', 'HCN', 'HCO', 'HCONH2', 'HCOOH']
['HCP', 'HCl', 'HF', 'HNC', 'HNO', 'HNS', 'HO2', 'HOCl', 'HOF', 'HOOH']
['He', 'Li', 'LiBH4', 'LiCN', 'LiCl', 'LiH', 'Mg', 'Mg2', 'N', 'N2']
['N2H2', 'N2H4', 'NCO', 'NH', 'NH2', 'NH2Cl', 'NH2F', 'NH2OH', 'NH3', 'NH3O']
['NO', 'NOCl', 'NP', 'Na', 'Na2', 'NaCN', 'NaCl', 'NaH', 'NaLi', 'Ne']
['O2', 'O3', 'OCl', 'OCl2', 'OF', 'OF2', 'OH', 'P', 'P2', 'P2H4']
['PH', 'PH2', 'PH2OH', 'PH3', 'PH3O', 'PS', 'S2', 'S2H2', 'SCl', 'SCl2']
['SF', 'SF2', 'SH', 'SH2', 'SO-trip', 'SO2', 'SiH3', 'SiH3Cl', 'SiH3F', 'SiH4']
['SiO']


We build a list with the molecules that, according to the Data analysis are not converged due to rmult in the alpha convergence. For these molecules we repeat the calculation rasing the highest rmult value to 12

In [16]:
mol_high_rmult = ['BeH','OH','C2H','CH3O','H2O-Li','FH-OH']

Run the dataset

In [39]:
options={'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-3,'atol':1.e-3,\
         'reference_data':HG_data,'data_directory':'Data'}

In [40]:
code=C.SystemCalculator(omp=omp,mpi_run=mpi_run,skip=True,verbose=False)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 16 /home/dalessandro/Applications/BigDFT/binaries/v1.8.4/install/bin/bigdft


In [41]:
# standard value used to perform the field convergence procedure
s_f = None
# used for BN,OH,PS,FH-OH
#s_f = 0.001

In [42]:
full_data = {}
for molecule in ['CH3O']:    #full_dataset: #full_dataset_split[2]+full_dataset_split[3]:
    for xc in ['lda_pw','pbe','pbe0']:
        psp = ['hgh_k'] if xc != 'pbe' else get_psp(molecule)
        for p in psp:
            print ''
            print molecule,xc,p
            gs_data=gs_study(molecule,(xc,p),code,options)
            full_data[(molecule,xc,p)]=alpha_study(gs_data,code,options,single_field=s_f)
            full_data[(molecule,xc,p)].update(gs_data)


CH3O lda_pw hgh_k
Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Convergence reached in Dataset "CH3O-lda_pw-hgh_k(GS)" for id " {'rmult': 5.0} "
Fetching results for id " {'f': 0.01} "
Fetching results for id " {'f': 0.005} "
Fetching results for id " {'f': 0.001} "
Fetching results for id " {'f': 0.0005} "
Fetching results for id " {'f': 0.0001} "
Convergence in field not reached
Fetching results for id " {'rmult': 5.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 6.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 7.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 8.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 9.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 10.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 11.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 12.0, 'f': 0.0001} "
('Convergence in domain not reached', LookupError('Convergence not reached, enlar

In [27]:
data_rmult = full_data['C2H','pbe','hgh_k']['dataset_rmult']
data_rmult.results

{0: array([24.58603, 24.58603, 27.5763 ]),
 1: array([24.87202, 24.87202, 27.8222 ]),
 2: array([25.533155, 25.533155, 27.8683  ]),
 3: array([25.009695, 25.009695, 27.87465 ]),
 4: array([25.21616, 25.21616, 27.87775]),
 5: array([25.244235, 25.244235, 27.87695 ]),
 6: array([25.035905, 25.035905, 27.878   ]),
 7: array([25.567155, 25.567155, 27.87815 ])}

In [28]:
results = data_to_save(full_data,options)
results

{'options': {'atol': 0.001,
  'data_directory': 'Data',
  'hgrids': 0.3,
  'rmult_fine': 9.0,
  'rtol': 0.001,
  'wf_convergence': 1e-06},
 ('C2H',
  'lda_pw',
  'hgh_k'): {'alpha_convergence': ({'f': 0.001, 'rmult': 7.0},
   [25.40469,
    25.40469,
    27.945699999999988]), 'input_gs': {'dft': {'gnrm_cv': 1e-06, 'hgrids': 0.3, 'ixc': '-001012', 'mpol': 1, 'nspin': 2, 'rmult': [5.0,
     9.0]},
   'psppar.C': {'Pseudopotential XC': 1},
   'psppar.H': {'Pseudopotential XC': 1}}, 'posinp': {'positions': [{'C': [0.0,
      0.0,
      0.0],
     'frag': ['molecule', '0']},
    {'C': [0.0, 0.0, -1.2165], 'frag': ['molecule', '0']},
    {'H': [0.0, 0.0, 1.0465], 'frag': ['molecule', '0']}],
   'units': 'angstroem'}},
 ('C2H', 'pbe', 'hgh_k'): {'alpha_convergence': None,
  'input_gs': {'dft': {'gnrm_cv': 1e-06,
    'hgrids': 0.3,
    'ixc': 11,
    'mpol': 1,
    'nspin': 2,
    'rmult': [5.0, 9.0]}},
  'posinp': {'positions': [{'C': [0.0, 0.0, 0.0], 'frag': ['molecule', '0']},
    {'C': [0.

In [16]:
#full_results = {}
full_results = yaml.load(open('full_results.yaml'))

In [19]:
#full_results['CH3NH2','pbe0','hgh_k']
full_results['OH','pbe','hgh_k']

{'alpha_convergence': ({'f': 0.001, 'rmult': 8.0},
  [6.8349435, 6.8349435, 9.32050000000001]),
 'input_gs': {'dft': {'gnrm_cv': 1e-06,
   'hgrids': 0.3,
   'ixc': 11,
   'mpol': 1,
   'nspin': 2,
   'rmult': [5.0, 9.0]}},
 'posinp': {'positions': [{'H': [0.0, 0.0, 0.0], 'frag': ['molecule', '0']},
   {'O': [0.0, 0.0, 0.9697], 'frag': ['molecule', '0']}],
  'units': 'angstroem'}}

In [17]:
full_results.update(results)

In [16]:
#full_results.keys()

In [18]:
with open('full_results.yaml', 'w') as outfile:
    yaml.dump(full_results, outfile, default_flow_style=False)

## Specific study analysis

Some studies of the dataset do not reach convergence with the standard workflow. For these studies we try to perform a dedicated analysis

In [8]:
import yaml
HG_data=yaml.load(open('../HG Dataset/hg_data.yaml'))

In [6]:
code=C.SystemCalculator(omp=omp,mpi_run=mpi_run,skip=True,verbose=False)

Initialize a Calculator with OMP_NUM_THREADS=1 and command mpirun -np 16 /home/dalessandro/Applications/BigDFT/binaries/v1.8.4/install/bin/bigdft


In [9]:
options={'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-3,'atol':1.e-3,\
         'reference_data':HG_data,'data_directory':'Data'}

In [23]:
molecule = 'PS'
xc = 'lda_pw'
psp = 'hgh_k'
study_data = {}

In [24]:
gs_data=gs_study(molecule,(xc,psp),code,options)

Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Fetching results for id " {'rmult': 7.0} "
Convergence reached in Dataset "PS-lda_pw-hgh_k(GS)" for id " {'rmult': 6.0} "


In [25]:
gs_data

{'dataset_gs': <BigDFT.Datasets.Dataset instance at 0x7f6037918248>,
 'input_gs': {'dft': {'gnrm_cv': 1e-06,
   'hgrids': 0.3,
   'ixc': '-001012',
   'mpol': 1,
   'nspin': 2,
   'rmult': [6.0, 9.0]},
  'psppar.P': {'Pseudopotential XC': 1},
  'psppar.S': {'Pseudopotential XC': 1}},
 'log_gs': <BigDFT.Logfiles.Logfile instance at 0x7f6035d19f38>,
 'molecule': 'PS',
 'posinp': {'positions': [{'S': [0.0, 0.0, 0.0], 'frag': ['molecule', '0']},
   {'P': [0.0, 0.0, 1.9], 'frag': ['molecule', '0']}],
  'units': 'angstroem'},
 'study': ('lda_pw', 'hgh_k')}

In [26]:
study_data[(molecule,xc,psp)]=alpha_study(gs_data,code,options)
study_data[(molecule,xc,psp)].update(gs_data)

Fetching results for id " {'f': 0.01} "
Fetching results for id " {'f': 0.005} "
Fetching results for id " {'f': 0.001} "
Fetching results for id " {'f': 0.0005} "
Fetching results for id " {'f': 0.0001} "
Convergence in field not reached
Fetching results for id " {'rmult': 6.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 7.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 8.0, 'f': 0.0001} "
Fetching results for id " {'rmult': 9.0, 'f': 0.0001} "
Convergence reached in Dataset "PS-lda_pw-hgh_k(domain)" for id " {'rmult': 8.0, 'f': 0.0001} "
('Relative difference in %', [-6.633780267896963, -6.633780267896963, -0.3419660837812266])


Collection of some results

In [27]:
# results for PS
data_field = study_data['PS','lda_pw','hgh_k']['dataset_field']
data_field.results

{0: array([35.24757 , 35.24757 , 63.926085]),
 1: array([35.14789, 35.14789, 63.69702]),
 2: array([35.11898, 35.11898, 63.625  ]),
 3: array([35.15776, 35.15776, 63.623  ]),
 4: array([36.774245, 36.774245, 63.622   ])}

In [31]:
# results for BN
data_field = study_data['BN','lda_pw','hgh_k']['dataset_field']
data_field.results

{0: array([18.146625, 18.146625, 31.24421 ]),
 1: array([18.097996, 18.097996, 31.08102 ]),
 2: array([18.109625, 18.109625, 31.02895 ]),
 3: array([18.426397, 18.426397, 31.0275  ]),
 4: array([20.277075, 20.277075, 31.027   ])}

In [23]:
# results for OH
data_field = study_data['OH','lda_pw','hgh_k']['dataset_field']
data_field.results

{0: array([6.7345085, 6.7345085, 9.088    ]),
 1: array([6.726453, 6.726453, 9.07523 ]),
 2: array([6.7660125, 6.766012 , 9.0712   ]),
 3: array([7.035406, 7.035406, 9.071   ]),
 4: array([7.4663755, 7.4663755, 9.071    ])}

In [12]:
def extract_statical_polarizability_linearity(label,rtol,atol,path,input_for_alpha,posinp,code,ref):
    
    #ints= [1e-3,2e-3,5e-3] # for BN
    ints= [5e-4,1e-3,2e-3,5e-3] # for OH,PS,CH3O
       
    results = {}
    results['alpha_values'] = {}
    rmult_fine=input_for_alpha['dft']['rmult'][1]
    rmult_list=map(float,range(int(input_for_alpha['dft']['rmult'][0]),13))
    for intensity in ints:
    
        # seek for the rmult convergence
        alpha_rmult=D.Dataset(label=label+'(domain)',run_dir=path)
        for rm in rmult_list:
            input_for_alpha.set_rmult(coarse=rm,fine=rmult_fine)
            alpha_rmult.append_run(id={'f':intensity,'rmult':rm},\
                runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,\
                            runner=code,intensity=intensity))
    
        try:
            import numpy as np
            data_final=alpha_rmult.seek_convergence(rtol=rtol,atol=atol)
            AuToA = 0.5291772085**3
            alpha_ref_ii = np.array(ref)/AuToA
            eps = 100.0 * (data_final[1]-alpha_ref_ii)/alpha_ref_ii
            print ('Relative difference in %',eps.tolist())
        except LookupError as e:
            print ('Convergence in domain not reached',e)
            data_final=None
        
        results['alpha_values'][intensity] = {'alpha_convergence': data_final, 'dataset_rmult': alpha_rmult}
    return results

def alpha_study(gs_data,code,options,single_field=None):
    """"
    Workflow for the convergence analysis for the computation of alpha.
    
    Args : 
       gs_data        : output of gs_study
       code (runner)  : instance of SystemCalculator
       options (dict) : dictionary with the computational options
    """
    import copy
    xc_conversion={'lda_pw': 'lda-SPW92', 'pbe': 'pbe', 'pbe0': 'pbe0'}
    rtol=options.get('rtol',1.e-2)
    atol=options.get('atol',1.e-3)
    reference_results=options.get('reference_data')
    datadir=options.get('data_directory','Data')
    study=gs_data['study']
    molecule=gs_data['molecule']
    posinp=gs_data['posinp']
    xc=study[0]
    ref_data=reference_results[molecule][xc_conversion[xc]]
    
    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,datadir)
    input_gs=copy.deepcopy(gs_data['input_gs'])
        
    data=extract_statical_polarizability_linearity(label,rtol,atol,path,input_gs,posinp,code,ref=ref_data)
    os.chdir(initial_dir)
    
    return data

In [10]:
molecule = 'CH3O'
xc = 'lda_pw'
psp = 'hgh_k'
study_data = {}

In [11]:
gs_data=gs_study(molecule,(xc,psp),code,options)

Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Convergence reached in Dataset "CH3O-lda_pw-hgh_k(GS)" for id " {'rmult': 5.0} "


In [33]:
#gs_data

In [13]:
study_data[(molecule,xc,psp)]=alpha_study(gs_data,code,options)
study_data[(molecule,xc,psp)].update(gs_data)

Fetching results for id " {'rmult': 5.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 6.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 7.0, 'f': 0.0005} "
Fetching results for id " {'rmult': 8.0, 'f': 0.0005} "
Convergence reached in Dataset "CH3O-lda_pw-hgh_k(domain)" for id " {'rmult': 7.0, 'f': 0.0005} "
('Relative difference in %', [375.17147810329845, 710.4895685149879, -0.7292205601945779])
Fetching results for id " {'rmult': 5.0, 'f': 0.001} "
Fetching results for id " {'rmult': 6.0, 'f': 0.001} "
Fetching results for id " {'rmult': 7.0, 'f': 0.001} "
Fetching results for id " {'rmult': 8.0, 'f': 0.001} "
Convergence reached in Dataset "CH3O-lda_pw-hgh_k(domain)" for id " {'rmult': 7.0, 'f': 0.001} "
('Relative difference in %', [166.68681275853152, 354.27658232270954, -0.7262546415666635])
Fetching results for id " {'rmult': 5.0, 'f': 0.002} "
Fetching results for id " {'rmult': 6.0, 'f': 0.002} "
Fetching results for id " {'rmult': 7.0, 'f': 0.002} "
Fetching resu

In [40]:
#study_data

In [14]:
# results for CH3O (lda hgh_k)
alphas = study_data['CH3O','lda_pw','hgh_k']['alpha_values']
alphas

{0.0005: {'alpha_convergence': ({'f': 0.0005, 'rmult': 7.0},
   array([164.30701, 164.5212 ,  26.7764 ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f2018967ea8>},
 0.001: {'alpha_convergence': ({'f': 0.001, 'rmult': 7.0},
   array([92.21621, 92.21356, 26.7772 ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f201786d758>},
 0.002: {'alpha_convergence': ({'f': 0.002, 'rmult': 7.0},
   array([56.0659  , 56.063725, 26.78055 ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f2017150560>},
 0.005: {'alpha_convergence': ({'f': 0.005, 'rmult': 8.0},
   array([34.38935, 34.40695, 26.83322])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f2016829248>}}

In [35]:
# results for PS (lda hgh_k)
alphas = study_data['PS','lda_pw','hgh_k']['alpha_values']
alphas

{0.0005: {'alpha_convergence': ({'f': 0.0005, 'rmult': 8.0},
   array([35.63018, 35.63018, 64.624  ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f6033694050>},
 0.001: {'alpha_convergence': ({'f': 0.001, 'rmult': 8.0},
   array([35.6316, 35.6316, 64.6276])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f6035bd4908>},
 0.002: {'alpha_convergence': ({'f': 0.002, 'rmult': 8.0},
   array([35.63728  , 35.63728  , 64.6419175])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f6035640950>},
 0.005: {'alpha_convergence': ({'f': 0.005, 'rmult': 8.0},
   array([35.67728, 35.67728, 64.744  ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f603502d518>}}

In [39]:
import numpy as np
np.allclose(alphas[0.001]['alpha_convergence'][1],alphas[0.002]['alpha_convergence'][1],rtol=1e-3,atol=1e-3)

True

In [48]:
# results for OH (pbe hgh_k)
alphas = study_data['OH','pbe','hgh_k']['alpha_values']
alphas

{0.0005: {'alpha_convergence': ({'f': 0.0005, 'rmult': 8.0},
   array([6.83494, 6.83494, 9.3203 ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7feffeb893b0>},
 0.001: {'alpha_convergence': ({'f': 0.001, 'rmult': 8.0},
   array([6.8349435, 6.8349435, 9.3205   ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7feffe495fc8>},
 0.002: {'alpha_convergence': ({'f': 0.002, 'rmult': 8.0},
   array([6.8357825, 6.8357825, 9.32125  ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7feffdde2a28>},
 0.005: {'alpha_convergence': ({'f': 0.005, 'rmult': 8.0},
   array([6.839758, 6.839758, 9.32648 ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7feffd5998c0>}}

In [50]:
import numpy as np
np.allclose(alphas[0.001]['alpha_convergence'][1],alphas[0.005]['alpha_convergence'][1],rtol=1e-3,atol=1e-3)

True

In [41]:
# results for OH (lda)
alphas = study_data['OH','lda_pw','hgh_k']['alpha_values']
alphas

{0.0005: {'alpha_convergence': ({'f': 0.0005, 'rmult': 8.0},
   array([6.939519, 6.939519, 9.3307  ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7ff000301488>},
 0.001: {'alpha_convergence': ({'f': 0.001, 'rmult': 8.0},
   array([6.9395655, 6.9395655, 9.33085  ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7ff0008df908>},
 0.002: {'alpha_convergence': ({'f': 0.002, 'rmult': 8.0},
   array([6.9407025, 6.9407025, 9.331625 ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7fefff9d43f8>},
 0.005: {'alpha_convergence': ({'f': 0.005, 'rmult': 8.0},
   array([6.944738, 6.944738, 9.33686 ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7fefff5a65f0>}}

In [43]:
import numpy as np
np.allclose(alphas[0.001]['alpha_convergence'][1],alphas[0.005]['alpha_convergence'][1],rtol=1e-3,atol=1e-3)

True

In [50]:
# results for BN
alphas = study_data['BN','lda_pw','hgh_k']['alpha_values']
alphas

{0.001: {'alpha_convergence': ({'f': 0.001, 'rmult': 6.0},
   array([18.18901, 18.18901, 31.29695])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f59f1a8a9e0>},
 0.002: {'alpha_convergence': ({'f': 0.002, 'rmult': 6.0},
   array([18.191185, 18.191185, 31.304575])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f59f1552488>},
 0.005: {'alpha_convergence': ({'f': 0.005, 'rmult': 7.0},
   array([18.219973, 18.219973, 31.3895  ])),
  'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7f59f0f99680>}}

In [51]:
import numpy as np
np.allclose(alphas[0.001]['alpha_convergence'][1],alphas[0.002]['alpha_convergence'][1],rtol=1e-3,atol=1e-3)

True

In [52]:
np.allclose(alphas[0.001]['alpha_convergence'][1],alphas[0.005]['alpha_convergence'][1],rtol=1e-3,atol=1e-3)

False

On the basis of this analysis we can state that a converged results is found for field 1em3

OH (in lda hgh_k) does not converge in the standard approach beacause the rmult used to perform the field convergence is too small